In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import sys
import yfinance as yf
import datetime
import bs4
import requests

In [72]:
fileDB = ''
start_date = datetime.datetime.today() - datetime.timedelta(weeks = 260)
end_date = datetime.datetime.today() - datetime.timedelta(days = 1)

cash = 10000

menu_options = ['Exit', 'Settings', 'Buy', 'Sell', 'See Portfolio', 'See Current Prices', 'Graph Past Performance']
command_numbers = list(range(len(menu_options)))
menu_options = dict(zip(command_numbers, menu_options))

pd.set_option("display.max_rows", None, "display.max_columns", None)

In [73]:
def extractYahoo_equity(asset):
    #call API routines specific to an asset
    equity = yf.download(asset,
                         start= start_date,
                         end= end_date,
                         progress=False,
                         auto_adjust=False)
    #drop extracts into ET fileDB
    equity.to_csv(fileDB + str(asset) + '.prices.csv')
    #equity.to_json(fileDB + str(asset) + 'prices.json')
    return equity

In [86]:
def extract_all_assets(assets_url):
    resp = requests.get(assets_url)
    soup = bs4.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)

    tickers = [s.replace('\n', '') for s in tickers]
    tickers = [s + ".AX" for s in tickers]
        
    equity = yf.download(tickers,
                         start= start_date,
                         end= end_date,
                         progress=False,
                         auto_adjust=False)
    #drop extracts into ET fileDB
    equity.to_csv(fileDB + 'all_prices.csv')
    #equity.to_json(fileDB + str(asset) + 'prices.json')
    return equity

In [91]:
def prep_assets(assets_url):
    print("Downloading and preparing financial assets...")

    all_prices = extract_all_assets(assets_url)

    all_prices = all_prices['Adj Close']

    all_prices = all_prices.dropna(axis = 1, how = 'all')
    
    return all_prices

In [76]:
#sets up the portfolio
def load_portfolio():
    
    global cash
    
    try:
        portfolio = pd.read_csv("portfolio.csv")
        print("Portfolio found and successfully loaded.")
        cash -= sum(portfolio['Total Amount'])
    except:
        portfolio = pd.DataFrame(data = [], columns = ['Stock','Price','Quantity','Total Amount'])
        print("No portfolio found. New portfolio created.")
        
    return portfolio

In [77]:
def print_actions(options):
    for key in options:
        print(key, ":", options[key])

In [102]:
def update_portfolio(portfolio, all_prices):
    for stock in portfolio['Stock']:
        portfolio['Price']['Stock'] = all_prices.tail(1)[stock].item()
        
    portfolio['Total Amount'] = portfolio['Price'] * portfolio['Quantity']
    
    return portfolio

In [134]:
def menu():
    
    print("Welcome, investor!")
    
    all_prices = prep_assets('https://en.wikipedia.org/wiki/S%26P/ASX_200')
    portfolio = update_portfolio(load_portfolio(), all_prices)
        
    while True:
        global cash
        
        portfolio_value = sum(portfolio['Total Amount'])
        print("You have $%.2lf in cash." % cash)
        print("Your portfolio value is $%.2lf." % portfolio_value)
        
        print_actions(menu_options)
        try:
            command = int(input("What do you want to do? "))
        except:
            continue
        
        if command == 0:
            print("Saving progress...")
            portfolio.to_csv("portfolio.csv")
            input("Goodbye! Enter any key to continue. ")
            sys.exit(0)
        elif command == 1:
            settings()
        elif command == 2:
            portfolio = buy_choose_ticker(all_prices)
        elif command == 3:
            portfolio = sell_choose_ticker(all_prices)
        elif command == 4:
            print(portfolio)
        elif command == 5:
            see_current_prices(all_prices)
        elif command == 6:
            graph_choose_ticker(all_prices)

In [85]:
def see_current_prices(all_prices):
    print("Most recent stock prices as at", str(datetime.datetime.today()))
    current = all_prices.tail(1)
    for stock in current.columns:
        print(stock, ":", round(current[stock].item(), 2))

In [80]:
def choose_ticker(all_prices):
    ticker_found = False
    while not ticker_found:
        ticker = input("Which stock ticker do you want to choose? Press Enter to return to main menu. ")
        if ticker == "":
            print("Returning to main menu.")
            return
        elif ticker in all_prices.columns:
            ticker_found = True
            return ticker
        else:
            print("Ticker not found. Try again.")    

In [89]:
def graph_choose_ticker(all_prices):

    ticker = choose_ticker(all_prices)
    
    if not ticker:
        return

    print("Which date do you want to start graphing from?")
    start_date = input("Enter your date in the form YYYY-MM-DD. ")

    print("Which date do you want to graph to?")
    end_date = input("Enter your date in the form YYYY-MM-DD. ")            

    try:
        graph_past_performance(all_prices, ticker, start_date, end_date)
    except:
        print("Something went wrong. Returning to main menu.")    
        
    return

In [82]:
def graph_past_performance(all_prices, ticker, start_date, end_date):
    plt.rcParams["figure.figsize"] = (12,8)
    plt.plot(all_prices[ticker][start_date: end_date])
    plt.title("Plot of {} from {} to {}".format(ticker, start_date, end_date))
    plt.xlabel("Date")
    plt.ylabel(ticker)
    plt.show()

In [100]:
def buy_choose_ticker(all_prices):
    
    see_current_prices(all_prices)
    
    ticker = choose_ticker(all_prices)    
    if not ticker:
        return
    
    return buy(all_prices, ticker, portfolio)  

In [137]:
if __name__ == "__main__":
    menu()

Welcome, investor!
No portfolio found. New portfolio created.
You have $nan in cash.
Your portfolio value is $0.00.
0 : Exit
1 : Settings
2 : Buy
3 : Sell
4 : See Portfolio
5 : See Current Prices
6 : Graph Past Performance
What do you want to do? 4
Empty DataFrame
Columns: [Stock, Price, Quantity, Total Amount]
Index: []
You have $nan in cash.
Your portfolio value is $0.00.
0 : Exit
1 : Settings
2 : Buy
3 : Sell
4 : See Portfolio
5 : See Current Prices
6 : Graph Past Performance
What do you want to do? 2
Most recent stock prices as at 2020-07-30 17:37:13.444978
A2M.AX : 19.45
ABC.AX : 2.26
ABP.AX : 2.69
AGL.AX : 17.05
ALL.AX : 25.83
ALQ.AX : 8.29
ALU.AX : 32.43
ALX.AX : 6.8
AMC.AX : 14.83
AMP.AX : 1.69
ANN.AX : 38.28
ANZ.AX : 18.45
APA.AX : 11.14
APD.AX : 0.51
APE.AX : 7.99
APT.AX : 67.77
APX.AX : 35.67
ARB.AX : 19.41
ASB.AX : 3.3
AST.AX : 1.78
ASX.AX : 83.24
AVH.AX : 6.39
AWC.AX : 1.62
AZJ.AX : 4.54
BAP.AX : 6.12
BEN.AX : 7.02
BGA.AX : 4.45
BHP.AX : 37.3
BIN.AX : 1.99
BKL.AX : 71.51
B

ValueError: cannot convert float NaN to integer

In [128]:
def buy(all_prices, ticker, portfolio):
    
    global cash
    
    price = all_prices.tail(1)[ticker].item()
    print(ticker, ":", round(price, 2))
    
    max_purchaseable = cash // price
    
    while True:
        
        print("How many do you want to buy? You can buy up to %d units." % max_purchaseable)
        try:
            number_purchased = int(input("Enter a non-numerical value to return to the main menu. "))
        except:
            print("Returning to main menu.")
            return portfolio
        
        if number_purchased > max_purchaseable:
            print("Insufficient funds.")
            
        else:
            break
            
    print("Purchased {0} of stock {1} for a total of ${2}.".format(number_purchased, ticker, round(number_purchased * price, 2)))
    
    if ticker not in portfolio['Stock']:
        new_stonk = pd.DataFrame([[ticker, price, number_purchased, number_purchased * price]])
        portfolio = portfolio.append(new_stonk, ignore_index = False)
    else:
        cash -= number_purchased * price
        portfolio[portfolio['Stock'] == ticker]['Quantity'] += number_purchased
        portfolio['Total Amount'] = portfolio['Price'] * portfolio['Quantity'] 

    return portfolio